In [ ]:
import earthaccess
import getpass
import os
from datetime import datetime
import geopandas as gpd
from shapely.geometry import box, Polygon, mapping
import pandas as pd
import folium
import webbrowser
from openpyxl.utils import get_column_letter

In [ ]:
# === Paths ===
IRAN_SHP_PATH = r"data/gadm41_IRN_0.shp"   # put shapefile here
OUTPUT_DIR    = r"output" 

def authenticate_earthdata():
    username = input("Enter your Earthdata username: ").strip()
    password = getpass.getpass("Enter your Earthdata password: ")
    try:
        os.environ['EARTHDATA_USERNAME'] = username
        os.environ['EARTHDATA_PASSWORD'] = password
        auth = earthaccess.login(strategy="environment")
        if auth.authenticated:
            print("Authentication successful!")
            print("=" * 70)
            return auth
        else:
            print("Authentication failed. Check username/password.")
            return None
    except Exception as e:
        print(f"Authentication failed: {e}")
        return None

def load_iran_shapefile():
    print("=" * 70)
    print("Loading Iran shapefile...")
    print("=" * 70)
    try:
        iran_gdf = gpd.read_file(IRAN_SHP_PATH)
        iran_gdf = iran_gdf.to_crs(epsg=4326)
        iran_polygon = iran_gdf.unary_union
        print("Iran shapefile loaded successfully.")
        return iran_gdf, iran_polygon
    except Exception as e:
        print(f"Failed to load Iran shapefile: {e}")
        return None, None

def search_modis_data(temporal_range, spatial_bbox):
    print(f"Product: MOD021KM")
    print(f"Temporal Range: {temporal_range[0]} to {temporal_range[1]}")
    print(f"Spatial Bbox: {spatial_bbox}")
    print("=" * 70)
    print("Searching...")
    print("=" * 70)
    try:
        results = earthaccess.search_data(
            short_name='MOD021KM',
            temporal=temporal_range,
            bounding_box=spatial_bbox
        )
        print(f"Found {len(results)} granules.")
        return results
    except Exception as e:
        print(f"Search failed: {e}")
        return []

def extract_granule_metadata(granule):
    metadata = {}
    try:
        umm = granule.get('umm', {})
        temporal = umm.get('TemporalExtent', {}).get('RangeDateTime', {})
        metadata['start_time'] = temporal.get('BeginningDateTime', 'N/A')
        metadata['end_time'] = temporal.get('EndingDateTime', 'N/A')
        spatial = umm.get('SpatialExtent', {}).get('HorizontalSpatialDomain', {})
        geometry = spatial.get('Geometry', {})
        if 'BoundingRectangles' in geometry:
            bounds = geometry['BoundingRectangles'][0]
            metadata['bbox'] = {
                'west': bounds.get('WestBoundingCoordinate'),
                'south': bounds.get('SouthBoundingCoordinate'),
                'east': bounds.get('EastBoundingCoordinate'),
                'north': bounds.get('NorthBoundingCoordinate')
            }
        elif 'GPolygons' in geometry:
            gpolygon = geometry.get('GPolygons', [])[0]
            boundary = gpolygon.get('Boundary', {})
            points = boundary.get('Points', [])
            metadata['polygon_points'] = [(p['Longitude'], p['Latitude']) for p in points]
        data_granule = umm.get('DataGranule', {})
        metadata['producer_granule_id'] = data_granule.get('ProducerGranuleId', 'N/A')
        related_urls = umm.get('RelatedUrls', [])
        for url_info in related_urls:
            if url_info.get('Type') == 'GET DATA':
                metadata['download_url'] = url_info.get('URL', 'N/A')
                break
        size_info = data_granule.get('ArchiveAndDistributionInformation', [{}])[0]
        metadata['size_mb'] = size_info.get('SizeInBytes', 0) / (1024 * 1024)
    except Exception as e:
        print(f"Warning: Error extracting metadata: {e}")
    return metadata

def create_granule_polygon(metadata):
    try:
        if 'polygon_points' in metadata:
            return Polygon(metadata['polygon_points'])
        elif 'bbox' in metadata:
            b = metadata['bbox']
            return box(b['west'], b['south'], b['east'], b['north'])
    except Exception as e:
        print(f"Warning: Could not create polygon: {e}")
    return None

def calculate_coverage_percentage(granule_poly, iran_poly):
    try:
        if granule_poly is None or iran_poly is None:
            return 0.0
        if not granule_poly.intersects(iran_poly):
            return 0.0
        intersection = granule_poly.intersection(iran_poly)
        coverage_pct = (intersection.area / iran_poly.area) * 100
        return round(coverage_pct, 2)
    except Exception as e:
        print(f"Warning: Error calculating coverage: {e}")
        return 0.0

def analyze_granules(results, iran_poly):
    #print("Analyzing granules...")
    granules_with_coverage = []
    for idx, granule in enumerate(results, 1):
        #print(f"\nGranule {idx}/{len(results)}")
        metadata = extract_granule_metadata(granule)
        #print(f"File: {metadata.get('producer_granule_id', 'N/A')}")
        #print(f"Start Time: {metadata.get('start_time', 'N/A')}")
        #print(f"End Time: {metadata.get('end_time', 'N/A')}")
        #print(f"Size: {metadata.get('size_mb', 0):.2f} MB")
        granule_poly = create_granule_polygon(metadata)
        if granule_poly:
            coverage = calculate_coverage_percentage(granule_poly, iran_poly)
            metadata['coverage_percentage'] = coverage
            #print(f"Intersects Iran: {'Yes' if coverage > 0 else 'No'}")
            #print(f"Coverage over Iran: {coverage}%")
            if coverage > 0:
                metadata['geometry'] = granule_poly
                granules_with_coverage.append(metadata)
        else:
            print("Spatial coverage: Unable to determine")
    return granules_with_coverage

def export_to_html_map(granules_data, iran_gdf, output_dir):
    timestamp = datetime.now().strftime('2023.01.01_2024.12.31')
    map_file = os.path.join(output_dir, f"MODIS_Iran_Coverage_Map_{timestamp}.html")

    m = folium.Map(location=[32.0, 53.0], zoom_start=5, tiles="CartoDB positron")

    folium.GeoJson(
        iran_gdf,
        name="Iran Boundary",
        style_function=lambda x: {
            'fillColor': 'lightgreen',
            'color': 'darkgreen',
            'weight': 3,
            'fillOpacity': 0.1
        },
        tooltip="Iran"
    ).add_to(m)

    for idx, g in enumerate(granules_data, 1):
        granule_poly = g['geometry']
        coverage = g['coverage_percentage']
        popup_text = f"Granule: {g['producer_granule_id']}<br>Coverage: {coverage}%<br>Size: {g['size_mb']:.2f} MB"

        if coverage > 90:
            color = "purple"
        elif coverage > 80:
            color = "red"
        elif coverage > 70:
            color = "orange"
        elif coverage > 50:
            color = "yellow"
        else:
            color = "blue"

        gj = folium.GeoJson(
            mapping(granule_poly),
            name=f"Granule {idx} ({coverage}%)",
            style_function=lambda x, c=color: {
                'fillColor': c,
                'color': c,
                'weight': 2,
                'fillOpacity': 0.3
            }
        )
        gj.add_child(folium.Popup(popup_text, max_width=300))
        gj.add_to(m)

    folium.LayerControl().add_to(m)
    m.save(map_file)
    abs_map_path = os.path.abspath(map_file)
    webbrowser.open(f'file://{abs_map_path}')
    print("=" * 70)
    print(f"HTML map saved to: {map_file}")

def export_to_excel(granules_data, output_dir):
    """Export granule data to Excel file with coverage > 50% only"""
    timestamp = datetime.now().strftime('2023.01.01_2024.12.31')
    excel_file = os.path.join(output_dir, f"MODIS_Iran_Results_{timestamp}.xlsx")
    
    # Convert to DataFrame
    df = pd.DataFrame(granules_data)
    
    # Filter to include only granules with coverage > 50%
    df = df[df['coverage_percentage'] > 50]
    
    # Sort by coverage percentage descending
    df = df.sort_values(by='coverage_percentage', ascending=False)
    
    # Select and order columns for export (exclude geometry)
    columns_to_export = ['producer_granule_id', 'start_time', 'end_time', 'size_mb', 'coverage_percentage']
    df_export = df[columns_to_export].copy()
    
    # Calculate summary statistics
    avg_coverage = df_export['coverage_percentage'].mean()
    total_size = df_export['size_mb'].sum()
    
    # Add summary row
    summary = pd.DataFrame([{
        'producer_granule_id': 'SUMMARY',
        'start_time': '',
        'end_time': '',
        'size_mb': total_size,
        'coverage_percentage': avg_coverage
    }])
    
    df_export = pd.concat([df_export, summary], ignore_index=True)
    
    # Write to Excel with openpyxl engine
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        df_export.to_excel(writer, sheet_name='MODIS Granules', index=False)
        
        # Auto-adjust column widths
        worksheet = writer.sheets['MODIS Granules']
        for idx, col in enumerate(df_export.columns, 1):
            max_length = max(
                df_export[col].astype(str).map(len).max(),
                len(str(col))
            ) + 2
            worksheet.column_dimensions[get_column_letter(idx)].width = max_length
    
    abs_path = os.path.abspath(excel_file)
    print(f"Excel file saved to: {excel_file}")
    
    # Attempt to open the file automatically
    try:
        os.startfile(abs_path)
    except AttributeError:
        # For non-Windows systems
        print(f"Please open the file manually: {abs_path}")
    except Exception as e:
        print(f"Could not auto-open file: {e}")
    
    return excel_file

def main():
    auth = authenticate_earthdata()
    if not auth:
        print("Exiting due to authentication failure.")
        return

    temporal_range = ('2018-01-01', '2025-11-10')
    spatial_bbox = (44.0, 25.0, 63.5, 40.0)

    results = search_modis_data(temporal_range, spatial_bbox)
    if not results:
        print("No granules found. Exiting.")
        return

    iran_gdf, iran_poly = load_iran_shapefile()
    if iran_poly is None:
        print("Could not load Iran shapefile.")
        return

    granules_with_coverage = analyze_granules(results, iran_poly)

    print("=" * 70)
    print("Summary")
    print("=" * 70)
    print(f"Total granules found: {len(results)}")
    print(f"Granules intersecting Iran: {len(granules_with_coverage)}")
    print(f"Granules saved to Excel (coverage > 50%): {len([g for g in granules_with_coverage if g['coverage_percentage'] > 50])}")

    if granules_with_coverage:
        export_to_html_map(granules_with_coverage, iran_gdf, OUTPUT_DIR)
        export_to_excel(granules_with_coverage, OUTPUT_DIR)

if __name__ == "__main__":
    main()